In [ ]:
import email
import pandas as pd
import pickle

from pypasta import Config
from pypasta.Repository.Mbox import Mbox
from pypasta.Repository.Mbox import PatchMail

In [ ]:
# !./pasta prepare_evaluation --review

In [ ]:
config = Config('linux')
mailbox = Mbox(config)

In [ ]:
# repo = config.repo

In [ ]:
with open('resources/linux/resources/patch_responses.pkl', 'rb') as f:
    patch_responses = pickle.load(f)

In [ ]:
patch_responses = pd.DataFrame(patch_responses)

In [ ]:
patch_responses['id'] = patch_responses.index 

In [ ]:
df_upstream = patch_responses[['id','cluster_id','patch_id','upstream']]

In [ ]:
# index = ['id','cluster_id','patch_id']
# df_upstream.set_index(index, inplace=True)

In [ ]:
# df_upstream = pd.melt(df_upstream['upstream'].apply(list).apply(pd.Series).reset_index(), 
#             id_vars=index,
#             value_name='upstream').sort_index()
# df_upstream.drop('variable', axis=1, inplace=True)

In [ ]:
df_response = patch_responses[['id','responses']]

In [ ]:
df_response.set_index('id', inplace=True)

In [ ]:
df_response = pd.melt(df_response['responses'].apply(pd.Series).reset_index(), 
            id_vars='id',
            value_name='responses').sort_index()
df_response.drop('variable', axis=1, inplace=True)

In [ ]:
patch_responses = pd.merge(df_response, df_upstream, on = 'id')

In [ ]:
def try_get_id(msg):
    try:
        return msg['resp_msg_id']
    except:
        return None

In [ ]:
patch_responses['resp_msg_id'] = patch_responses['responses'].map(try_get_id)

In [ ]:
patch_responses['mailing_list'] = patch_responses['resp_msg_id'].map(lambda x: mailbox.get_lists(x))

In [ ]:
patch_responses['email'] = patch_responses['resp_msg_id'].map(lambda x: mailbox.get_messages(x))

In [ ]:
def try_get_date(email):
    try:
        return pd.to_datetime(email.get('Date'), utc=True)
    except:
        None

patch_responses['date'] = patch_responses['email'].map(try_get_date)

In [ ]:
# def get_patch_object(mail):
#     try:
#         return PatchMail(mail, mail['Message-Id'])
#     except:
#         return None

In [ ]:
# patch_responses['Patch'] = patch_responses['email'].apply(get_patch_object)
# patch_responses.head(2)

In [ ]:
patch_responses['date'] = patch_responses['email'].map(try_get_date)

In [ ]:
patch_responses.drop('email', inplace=True)

In [ ]:
patch_responses.to_csv('patch_responses.csv')